Projet réalisé par Fréjoux Gaëtan et Niord Mathieu.  
Projet portant sur l'apprentissage par renforcement.

## 1. Développement d'un jeu

* *Créer une fonction qui permet de simuler le plateau en positionnant notamment les différents éléments (case départ, fin, dragons)*

In [204]:
BOARD = [   
        'S', ' ', ' ', ' ', 
        'D', ' ', 'D', ' ', 
        ' ', ' ', ' ', 'D', 
        ' ', 'D', ' ', 'J'
    ]

Etant donné que le plateau ne change pas, nous avons décidé de le représenter directement avec une matrice sans utiliser une fonction.

* *Créer une fonction qui permet de simuler l'interaction entre l'agent et son environnement*

In [205]:
UP = 0              # GO UP
RIGHT = 1           # GO RIGHT
DOWN = 2            # GO DOWN
LEFT = 3            # GO LEFT

JAIL_REWARD = 1     # Reward for the jail cell
DRAGON_REWARD = -1  # Reward for the dragon cell
NORMAL_REWARD = 0   # Reward for the normal cell

NB_GAME = 2000      # Number of games
ALPHA = 0.81        # Learning rate
GAMMA = 0.96        # Discount factor

In [206]:
def application_action(action, position, space):
    def type_to_reward(type):
        if type == 'J':
            return JAIL_REWARD
        elif type == 'D':
            return DRAGON_REWARD
        else:
            return NORMAL_REWARD

    if (action == UP):
        if (position - 4 >= 0):
            position -= 4
    elif (action == RIGHT):
        if (position % 4 != 3):
            position += 1
    elif (action == DOWN):
        if (position + 4 < 16):
            position += 4
    elif (action == LEFT):
        if (position % 4 != 0):
            position -= 1
    else:
        print('Error : action not recognized')
    if (space[position] == 'D'):
        return 0, -1, True
    elif (space[position] == 'J'):
        return 0, JAIL_REWARD, True
    else:
        return position, NORMAL_REWARD, False

## 2. Développement du Q-learning

* *Créer une fonction nommé **choose_action** qui applique la stratégie epsilon-greedy pour choisir l'action à effectuer selon l'état de l'agent (ou sa position)*

In [207]:
import numpy as np
import random
def choose_action(state, epsilon, mat_q):
    if random.random() < epsilon:
        return random.randint(0, 3)
    else:
        return np.argmax(mat_q[state])

* *Créer une fonction nommé **onestep** qui applique une itération de l'algorithme Q-learning*

In [208]:
def onestep(mat_q, state, epsilon):
    action = choose_action(state, epsilon, mat_q)
    new_state, reward, is_end = application_action(action, state, BOARD)
    mat_q[state, action] = mat_q[state, action] + ALPHA * \
        (reward + GAMMA * np.max(mat_q[new_state]) - mat_q[state, action])
    return new_state, reward, is_end, mat_q

* *Tester votre algorithme avec des récompenses R = −1, 0, 1. Nous proposons d’étudier le paramétrage suivant : α = 0.81, γ = 0.96.*

In [209]:
import matplotlib.pyplot as plt
NB_GAME = 10000
MAX_STEP = 100


def run(q, epsilon, max_step):
    """
    Run a game
    """
    state = 0
    is_end = False
    nb_step = 0
    reward = 0
    while not is_end and nb_step < max_step:
        state, reward, is_end, q = onestep(q, state, epsilon)
        nb_step += 1
    return nb_step, reward, q


def train(nb_game, max_step):
    """
    Train the agent
    """
    nb_steps = []
    q = np.zeros((16, 4))
    minimum = 0
    success_number_of_steps = []
    for i in range(nb_game):
        nb_step, reward, q = run(q, nb_game / (i + nb_game), max_step)
    return q


q = train(NB_GAME, MAX_STEP)
print(q)


# show the board with the best action for each state add the legend for each action
#plt.imshow(np.argmax(q, axis=1).reshape(4, 4))
# plt.colorbar()
# plt.show()


[[3.603157   3.75328854 2.603157   3.603157  ]
 [3.75328854 3.603157   3.90967556 3.603157  ]
 [3.603157   3.45903072 2.603157   3.75328854]
 [3.45903072 3.45903072 3.32066949 3.603157  ]
 [0.         0.         0.         0.        ]
 [3.75328854 2.603157   4.07257871 2.603157  ]
 [0.         0.         0.         0.        ]
 [3.45903072 3.32066949 2.603157   2.60315699]
 [2.603157   4.07257871 3.75328854 3.90967556]
 [3.90967556 4.24226949 2.603157   3.90967556]
 [2.603157   2.603157   4.41903072 4.07257871]
 [0.         0.         0.         0.        ]
 [3.90967556 2.603157   3.75328854 3.75328854]
 [0.         0.         0.         0.        ]
 [4.24226949 4.603157   4.41903072 2.603157  ]
 [0.         0.         0.         0.        ]]


* *Etudier la table Q*

On observe suite à l'entrainement que la table Q est remplie de valeurs.
On remarque tout d'abord que les cases comprenants des dragons ou la case de fin sont remplis de **0**. Cela est normal car il **n'est pas possible** de se déplacer sur ces cases.
Enfin, on remarque que pour chaque état, la meilleur action possible est celle ayant la plus grande valeur.

*Jouer une partie avec la politique optimale liée à la table et commenter le parcours*

In [210]:
nb_step, _, _ = run(q, 0, 100)

print("Number of steps to reach the jail : ", nb_step)


Number of steps to reach the jail :  6


On observe qu'avec un entrainement assez conséquent, on obtiens une politique optimale qui permet de gagner la partie en 6 coups.

In [211]:
DRAGON_REWARD = -10  # Reward for the dragon cell
NORMAL_REWARD = 0   # Reward for the normal cell
JAIL_REWARD = 200     # Reward for the jail cell

ALPHA = 0.81        # Learning rate
GAMMA = 0.96        # Discount factor

#TODO
q = train(10000, MAX_STEP)
print(q)

[[720.63139903 750.65770732 719.63139903 720.63139903]
 [750.65770732 720.63139903 781.93511179 720.63139903]
 [720.63139903 691.80614307 719.63139903 750.65770732]
 [691.80614307 691.80614307 690.84614307 720.63139903]
 [  0.           0.           0.           0.        ]
 [750.65770732 719.63139903 814.51574145 719.63139903]
 [  0.           0.           0.           0.        ]
 [691.80614304 690.84614304 719.63139903 719.63139888]
 [719.63139903 814.51574145 750.65770732 781.93511179]
 [781.93511179 848.45389734 719.63139903 781.93511179]
 [719.63139903 719.63139903 883.80614307 814.51574145]
 [  0.           0.           0.           0.        ]
 [781.93511179 719.63139903 750.65770732 750.65770732]
 [  0.           0.           0.           0.        ]
 [848.45389734 920.63139903 883.80614307 719.63139903]
 [  0.           0.           0.           0.        ]]


*Faites évoluer votre algorithme en modifiant les récompenses pour gagner en efficacité.*  

In [212]:
#TODO

*Etudier la table Q.* 

In [213]:
#TODO

*Jouer une partie avec la politique optimale liée à la table et commenter le parcours.*

In [214]:
#TODO

TODO COMMENTAIRE

*Comparer avec la première politique*

TODO COMPARAISON

## 3. Deep Q-learning

*Modifier la fonction **choose action(state,epsilon,modele)** qui va choisir dans certains cas l'action à appliquer avec :  
**Sortie Q = model.predict(np.array([vec etat]))***

In [215]:
import tensorflow as tf
import tensorflow.keras as keras

model = keras.Sequential([
    keras.layers.Dense(16, activation='relu', input_shape=(16,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(4, activation='linear')
])

#new version with predict model.predict(np.array([vec etat]))
def choose_action(state, epsilon, model):
    if random.random() < epsilon:
        return random.randint(0, 3)
    else:
        return np.argmax(model.predict(np.array([state])))

*la procédure permettant de mettre à jour les poids du réseau grâce à la différentiation automatique **tf.GradientTape()***

In [216]:
def update_weights(state, action, reward, new_state, model):
    target = reward + GAMMA * np.max(model.predict(np.array([new_state])))
    target_vec = model.predict(np.array([state]))
    target_vec[0][action] = target
    model.fit(np.array([state]), target_vec, epochs=1, verbose=0)

*Tester votre algorithme avec des récompenses R = −20, −1, 100*

In [217]:
#TODO

*Jouer une partie avec la politique optimale liée à la table et commenter le parcours*

In [218]:
#TODO

TODO COMMENTAIRE

*Faites évoluer votre algorithme en introduisant un second réseau*